In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import torch

import json
from pathlib import Path
from copy import deepcopy

import pyciemss.visuals.vega as vega
from pyciemss.utils import get_tspan
from pyciemss.utils.interface_utils import convert_to_output_format


In [37]:
# Load cached data...
tspan = get_tspan(1, 50, 500)

root = Path("..")/"test"/"data/"
prior_samples = convert_to_output_format(vega.tensor_load(root/"prior_samples.json"), tspan)
intervened_samples = vega.tensor_load(root/"intervened_samples.json")
observed_trajectory = convert_to_output_format(vega.tensor_load(root/"observed_trajectory.json"), tspan)

observed_points = (observed_trajectory
    .rename(columns={"Rabbits_sol": "Rabbits Samples"})
    .drop(columns=["Wolves_sol", "alpha_param", "beta_param", "delta_param", "gamma_param"])
    .iloc[::10]
)


In [96]:
nice_labels={"Rabbits_sol": "Rabbits", "Wolves_sol": "Wolves"}

schema = vega.trajectories(prior_samples, 
                           subset=".*_sol", 
                           relabel=nice_labels,
                           points=observed_points
                           )
schema = vega.pad(schema, 5)

vega.save_schema(schema, "_schema.json")
vega.ipy_display(schema)

In [65]:
wolves = prior_samples.set_index(["sample_id", "timepoint_id"])["Wolves_sol"].loc[1].rename("Wolves Example")
rabbits = prior_samples.set_index(["sample_id", "timepoint_id"])["Rabbits_sol"].loc[1].rename("Rabbits Example")
traces = pd.DataFrame([wolves, rabbits]).T
nice_labels={"Rabbits_sol": "Rabbits", "Wolves_sol": "Wolves"}
colors = {"Rabbits": "#aec7e8", 
          "Wolves": "#fdae6b",  
          "Wolves Example": "#ff7f0e",
          "Rabbits Example": "#1f77b4",
          "trash": "black", 
          "gamma": None}

schema = vega.trajectories(prior_samples,
                           traces=traces, 
                           relabel=nice_labels,
                           colors=colors,
                           markers={"Confident": 100, "Uncertain": 250}
                           )
schema = vega.pad(schema, 5)
vega.save_schema(schema, "_schema.json")
vega.ipy_display(schema)